In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Drop irrelevant features
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
##Encode categorical variable

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
##One Hot Encode - Geography column
from sklearn.preprocessing import OneHotEncoder

In [6]:
onehotencoder = OneHotEncoder()
geo_encoder = onehotencoder.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehotencoder.get_feature_names_out(['Geography']))

In [8]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [9]:
# save the encoders

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehotencoder_geo.pkl','wb') as file:
    pickle.dump(onehotencoder,file)

In [10]:
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']


In [11]:
#split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#scale the features

scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [12]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [13]:
#ANN Implementation
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import tensorboard
import datetime

In [14]:
model=Sequential([
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
        Dense(32,activation='relu'),
        Dense(1)
])

c:\Users\Neha Singhal\OneDrive\Desktop\Python Projects\Ann\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
##compile the model
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=['mae'])

In [17]:
#Setup the tensorboard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
tensorflow_callback= TensorBoard(log_dir=log_dir,histogram_freq=1)
##setup EarlyStopping
earlystopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
##training the model
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                    callbacks=[tensorflow_callback,earlystopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99909.2812 - mae: 99909.2812 - val_loss: 98429.3438 - val_mae: 98429.3438
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 100095.4453 - mae: 100095.4453 - val_loss: 96264.0078 - val_mae: 96264.0078
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 96687.6172 - mae: 96687.6172 - val_loss: 90874.7656 - val_mae: 90874.7656
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 90589.8672 - mae: 90589.8672 - val_loss: 82397.3203 - val_mae: 82397.3203
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 81554.0703 - mae: 81554.0703 - val_loss: 72289.9219 - val_mae: 72289.9219
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 71335.8516 - mae: 71335.8516 - val_loss: 63064.2695 - val_mae: 63064.2695
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 61756.4258 - mae: 61756.4258 - val_loss: 56314.1758 - val_mae: 56314.1758
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

In [18]:
model.save('regmodel.h5')

In [22]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
%tensorboard --logdir regressionlogs/fit/

Reusing TensorBoard on port 6009 (pid 22852), started 0:00:31 ago. (Use '!kill 22852' to kill it.)

In [24]:
#evaluate model on test data

test_loss,test_mae = model.evaluate(X_test,y_test)
print(f"test mae is {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51195.6172 - mae: 51195.6172
test mae is 50256.71875
